# Prepare Gibbs-Duhem Simulations

In [4]:
import os, sys 
import numpy as np

In [5]:
settings=dict()

settings['--step']=250
settings['--initial_equilibration_steps']=500000
settings['--steps_per_sim']=500000

settings['--percent_equilibration']=25.0
settings['--lmp_exe']='$LAMMPS_INTEL'
settings['--run_cmd']='"export OMP_NUM_THREADS=2;mpirun -n 48"'
settings['--lmp_options']='" -sf omp -partition 2x24 "'

settings['--root_fold']='./'

settings_T=settings.copy()
settings_T['--integration_variable']= 'T'
settings_T['--step']=5

settings_II=settings.copy()
settings['--initial_equilibration_steps']=1000000
settings['--steps_per_sim']=500000

In [6]:
def PrepSingleSim(setttings_temp,dir_name,left_phase,right_phase,pm,tm,pmin,pmax):
    dir_name=dir_name+'/'
    # Create simulation dir
    root_folder=dir_name+'/'+str(pm)+'atm'+'_'+str(tm)+'K'
    os.makedirs(dir_name, exist_ok=True)#s.system('mkdir  {}'.format(dir_name))
    os.makedirs(dir_name+'/Initial', exist_ok=True)
    os.makedirs(root_folder, exist_ok=True)

    os.system('cp integrate.py ' + dir_name)
    initial='Initial/'
    os.system('cp -r InitialConfigurations/{} {}{}'.format(left_phase,dir_name, initial) )
    os.system('cp -r InitialConfigurations/{} {}{}'.format(right_phase,dir_name, initial))

    
    
    # Setup command for running

    setttings_temp['--end_variable']=str(pmax)
    setttings_temp['--initial_point_folder']='../'+initial
    setttings_temp['--left']=left_phase
    setttings_temp['--right']=right_phase
    setttings_temp['--initial_TP']=str(tm)+' '+str(pm) 
    arguments=' '.join(['{} {}'.format(t,setttings_temp[t]) for t in setttings_temp])
    
    # Upwards
    arguments=' '.join(['{} {}'.format(t,setttings_temp[t]) for t in setttings_temp])
    command = 'python3 ../integrate.py {}\n'.format(arguments) 
    
    #Downwards
    setttings_temp['--end_variable']=str(pmin)
    arguments=' '.join(['{} {}'.format(t,setttings_temp[t]) for t in setttings_temp])
    command += 'python3 ../integrate.py {}\n'.format(arguments) 
    
    with open(root_folder+'/script.sh','w') as fp:
        fp.write(command)
    

### IceIh-Liquid

In [11]:
for PT in np.loadtxt('../IceIh-Liquid/4-BiasedCoexistence/P_T.dat'):
    print(PT)
    PrepSingleSim(settings,'IceIh_Liquid/','IceIh','Liquid',PT[0],PT[1],-5000,4000)

[  0.  269.5]
[1000.   259.3]
[2000.   246.3]
[2500.   236.5]
[3000.   217.3]


## IceII-Liquid

In [5]:
for PT in np.loadtxt('../../../TMP_SIMULATIONS/IceII_liquidv2/5-BiasedCoexistencev2/P_T.dat'):
    print(PT)

    PrepSingleSim(settings_II,'IceII_Liquid_2nd/','IceII','Liquid',PT[0],PT[1],0,8000)

[1.000e+03 2.368e+02 5.800e+00 2.429e-02]
[2.000e+03 2.396e+02 4.600e+00 1.902e-02]
[5.000e+03 2.486e+02 4.000e+00 1.610e-02]


## IceIII-Liquid

In [1]:
for PT in np.loadtxt('../../../TMP_SIMULATIONS/IceIII_liquid_v2/5-BiasedCoexistence//P_T.dat'):
    print(PT)
    PrepSingleSim(settings,'IceIII_Liquid_2nd/','IceIII','Liquid',PT[0],PT[1],0,10000)

NameError: name 'np' is not defined

## IceV-Liquid

In [ ]:
for PT in np.loadtxt('../IceV-Liquid/4-BiasedCoexistence/P_T.dat'):
    print(PT)
    PrepSingleSim(settings,'IceV_Liquid/','IceV','Liquid',PT[0],PT[1],2000,12000)

## IceVI-Liquid

In [6]:
for PT in np.loadtxt('../../../TMP_SIMULATIONS/IceVI_liquid_v2/5-BiasedCoexistence//P_T.dat'):
    print(PT)
    PrepSingleSim(settings,'IceVI_Liquid_2nd/','IceVI','Liquid',PT[0],PT[1],6000,13000)

[8.000e+03 2.605e+02 4.000e+00 1.542e-02]
[9.000e+03 2.683e+02 4.400e+00 1.625e-02]
[1.000e+04 2.751e+02 4.800e+00 1.752e-02]
[1.200e+04 2.876e+02 5.000e+00 1.751e-02]


## IceVI-IceV

In [9]:
with open('../FullPhaseDiagram/triple_points.dat','r') as fp:
    lines=fp.readlines()
    
    for line in lines:
        PT=line.split()
        if 'Liquid IceV IceVI' in line:
            PrepSingleSim(settings_T,'IceVI_IceV_2nd/','IceVI','IceV',float(PT[-1]),float(PT[-2]),50,280)

## IceIII IceV

In [10]:
with open('../FullPhaseDiagram/triple_points.dat','r') as fp:
    lines=fp.readlines()
    
    for line in lines:
        PT=line.split()
        if 'Liquid IceIII IceV' in line:
            PrepSingleSim(settings_T,'IceV_IceIII_2nd/','IceV','IceIII',float(PT[-1]),float(PT[-2]),50,280)

## IceII IceIh

In [5]:
with open('../FullPhaseDiagram/triple_points.dat','r') as fp:
    lines=fp.readlines()
    
    for line in lines:
        PT=line.split()
        if 'Liquid IceIh IceII' in line:
            PrepSingleSim(settings_T,'IceII_IceIh_2nd/','IceII','IceIh',float(PT[-1]),float(PT[-2]),0,250)

## IceIII IceIh

In [7]:
with open('../FullPhaseDiagram/triple_points.dat','r') as fp:
    lines=fp.readlines()
    
    for line in lines:
        PT=line.split()
        if 'Liquid IceIII IceIh' in line:
            PrepSingleSim(settings_T,'IceIh_IceIII_2nd/','IceIh','IceIII',float(PT[-1]),float(PT[-2]),0,250)

## IceII IceIII

In [14]:
with open('../FullPhaseDiagram/triple_points.dat','r') as fp:
    lines=fp.readlines()
    
    for line in lines:
        PT=line.split()
        if 'IceII IceIII IceV' in line or 'IceII IceIh IceIII' in line:
            PrepSingleSim(settings,'IceII_IceIII_2nd/','IceII','IceIII',float(PT[-1]),float(PT[-2]),12000,-2000)

## IceII IceV

In [11]:
with open('../FullPhaseDiagram/triple_points.dat','r') as fp:
    lines=fp.readlines()
    
    for line in lines:
        PT=line.split()
        if 'IceII IceIII IceV' in line:
            PrepSingleSim(settings_T,'IceII_IceV_2nd/','IceII','IceV',float(PT[-1]),float(PT[-2]),300,0)

## IceII IceVI

In [15]:
with open('../FullPhaseDiagram/triple_points.dat','r') as fp:
    lines=fp.readlines()
    
    for line in lines:
        PT=line.split()
        if 'IceII IceV IceVI' in line:
            PrepSingleSim(settings_T,'IceII_IceVI_2nd/','IceII','IceVI',float(PT[-1]),float(PT[-2]),300,0)

## IceII IceVI v2

In [10]:
with open('../FullPhaseDiagram/triple_points.dat','r') as fp:
    lines=fp.readlines()
    
    for line in lines:
        PT=line.split()
        if 'Liquid IceII IceVI' in line:
            PrepSingleSim(settings_T,'IceII_IceVI_v2/','IceII','IceVI',float(PT[-1]),float(PT[-2]),300,0)

## IceII IceV

In [4]:
with open('../FullPhaseDiagram/triple_points.dat','r') as fp:
    lines=fp.readlines()
    
    for line in lines:
        PT=line.split()
        if 'Liquid IceII IceV' in line:
            PrepSingleSim(settings_T,'IceII_IceV_2nd/','IceII','IceV',float(PT[-1]),float(PT[-2]),300,0)

In [ ]:
!jupyter nbconvert --to script PrepareGibbsDuhem.ipynb